In [15]:
import networkx as nx
import random
import itertools
import time

In [16]:
def load_graph_from_file(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [17]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [18]:
def PocetnoRjesenje(G):
    B = set()
    while not is_resolving_set(B, G):
        v = random.choice(list(G.nodes))
        B.add(v)
    return B

In [19]:
def Nk(B, k, G):
    V = set(G.nodes)
    if k <= len(B):
        # Izbor k elemenata iz komplementa skupa B
        complement = V - B
        new_sets = []
        for comb in itertools.combinations(complement, k):
            new_set = B - set([random.choice(list(B))])  # Brisanje jednog elementa iz B
            new_set.update(comb)  # Dodavanje novih elemenata iz komplementa
            new_sets.append(new_set)
        return new_sets
    return []

In [20]:
def DeleteLast(B):
    B0 = B.copy()  
    last_element = list(B0)[-1]
    B0.remove(last_element) 
    return B0  

In [21]:
def Shaking(B, k, G):
    neighbors = Nk(B, k, G)
    return random.choice(neighbors) if neighbors else B

In [22]:
def ObjF(B, G):
    distances = {v: [nx.shortest_path_length(G, v, u) for u in B] for v in G.nodes}
    count = 0
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            count += 1
    return count

In [23]:
def Compare(B0, B00, pmove,G):
    if  len(B00) < len(B0) or ObjF(B00,G) < ObjF(B0,G):
        return True
    elif len(B00) == len(B0) and ObjF(B00,G) > ObjF(B0,G):
        return False
    elif len(B00) == len(B0) and ObjF(B00,G) == ObjF(B0,G) and random.random() < pmove:
        return True
    return False

In [24]:
def LexSort(G, B):
    distances = {v: tuple(nx.shortest_path_length(G, v, u) for u in B) for v in G.nodes}
    sorted_vertices = sorted(G.nodes, key=lambda v: distances[v])
    return sorted_vertices, distances

In [25]:
def IdentifyBlocks(sorted_vertices, distances):
    # Identifikuje blokove (grupe sa istim metričkim koordinatama)
    blocks = []
    current_block = [sorted_vertices[0]]

    for v in sorted_vertices[1:]:
        if distances[v] == distances[current_block[0]]:
            current_block.append(v)
        else:
            if len(current_block) > 1:
                blocks.append(current_block)
            current_block = [v]

    if len(current_block) > 1:
        blocks.append(current_block)

    return blocks

In [26]:
def LocalSearch(B, B00, G):
    improved = True

    while improved:
        improved = False
        objval = ObjF(B00, G)

        for vr in B00:  
           
            z = {v: 0 for v in set(G.nodes) - B00}
             
            B00_minus_vr = B00 - {vr}

            sorted_vertices, distances = LexSort(G, B00_minus_vr)
            blocks = IdentifyBlocks(sorted_vertices, distances)           

            # Ažuriranje z[v] na osnovu blokova (korak 8-12)
            for block in blocks:
                # for p, q in itertools.combinations(block, 2):
                for p in block:
                    for q in block:
                        if(q > p):
                            for v in set(G.nodes) - B00:
                                if nx.shortest_path_length(G, p, v) == nx.shortest_path_length(G, q, v):
                                    z[v] += 1

            # print(z)
            # Pronalazak minimalnog z[v] (korak 13)
            vmin = min(z, key=z.get)

            # Korak 14-18: Ako nađemo poboljšanje DOBRO
            if z[vmin] == 0:
                B = (B00 | {vmin}) - {vr}
                B00 = DeleteLast(B)
                objval = ObjF(B00, G)
                improved = True

            # Korak 19-23: Ako je z[vmin] bolje od trenutnog objval DOBRO
            elif z[vmin] < objval:
                B00 = (B00 | {vmin}) - {vr}
                objval = z[vmin]
                improved = True
    return B

In [27]:
def VNS(kmin, kmax, itermax, pmove, G,max_time):
    B = PocetnoRjesenje(G)
    B0 = DeleteLast(B)
    iter = 0
    start_time = time.time() 
    k = kmin
    print('Pocetna ',B,B0)
    while itermax > iter:
        iter +=1
              
        # print('B0',B0)
        B00 = Shaking(B0, k, G)
        # print('prije',B00)
        B00 = LocalSearch(B, B00, G)
        
        # print('poslije',B00)
        if Compare(B0, B00, pmove, G):
        # if Compare(B00, B0, pmove, G):
            B0 = B00  
        else:
            if k < kmax:
                k += 1
            else:
                k = kmin
        # if iter == 0: 
        print(f"Iteracija {iter}, najbolji skup {B0}")
    return B0

In [28]:
# G = nx.Graph()
# G.add_edges_from([(1,2),(1,3),(1,5),(1,6),(2,4),(2,6),(3,5),(3,4),(4,5),(4,6),(5,6)])

G = load_graph_from_file('grafovi\gcol1.txt')

# Parametri za VNS
kmin = 1
kmax = 3
itermax = 100
pmove = 0.2

# Pozivanje funkcije VNS
result = VNS(kmin, kmax, itermax, pmove, G,10)
print("Najbolji rešavajući skup:", result)
print(is_resolving_set(result,G))

<>:4: SyntaxWarning: invalid escape sequence '\g'
<>:4: SyntaxWarning: invalid escape sequence '\g'
C:\Users\pc\AppData\Local\Temp\ipykernel_12436\1624061564.py:4: SyntaxWarning: invalid escape sequence '\g'
  G = load_graph_from_file('grafovi\gcol1.txt')


Pocetna  {99, 100, 35, 38, 39, 73, 41, 43, 10, 45, 42, 60, 95} {99, 100, 35, 38, 39, 73, 41, 43, 10, 45, 42, 60}
Iteracija 1, najbolji skup {1, 98, 3, 67, 2, 39, 41, 76, 45}
Iteracija 2, najbolji skup {1, 98, 3, 67, 2, 39, 41, 76, 45}
Iteracija 3, najbolji skup {1, 98, 3, 67, 2, 39, 41, 76, 45}
Iteracija 4, najbolji skup {1, 98, 3, 67, 2, 39, 41, 76, 45}
Iteracija 5, najbolji skup {98, 3, 67, 39, 41, 76, 45, 28, 30}
Iteracija 6, najbolji skup {98, 3, 67, 39, 41, 76, 45, 28, 30}
Iteracija 7, najbolji skup {1, 2, 8, 76, 45, 13, 46, 20}
Iteracija 8, najbolji skup {1, 97, 3, 9, 76, 13, 46, 15, 20}
Iteracija 9, najbolji skup {1, 5, 10, 76, 12, 45, 19, 88}
Iteracija 10, najbolji skup {33, 10, 44, 45, 12, 17, 23, 88, 94}
Iteracija 11, najbolji skup {37, 39, 10, 44, 45, 12, 17, 82, 23}
Iteracija 12, najbolji skup {37, 39, 10, 44, 45, 12, 17, 82, 23}
Iteracija 13, najbolji skup {37, 39, 10, 44, 45, 12, 17, 82, 23}
Iteracija 14, najbolji skup {37, 39, 10, 44, 45, 12, 17, 82, 23}
Iteracija 15, na